In [1]:
import pandas as pd
import feedparser
import nltk
import warnings
from datetime import datetime
from pathlib import Path
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')
from googletrans import Translator
translator = Translator()
#bedrijf830@gmail.com
#Kom je nog?

import csv

In [4]:
sia = SentimentIntensityAnalyzer()

import csv

# stock market lexicon
stock_lex = pd.read_csv('lexicon_data/stock_lex.csv')
stock_lex['sentiment'] = (stock_lex['Aff_Score'] + stock_lex['Neg_Score'])/2
stock_lex = dict(zip(stock_lex.Item, stock_lex.sentiment))
stock_lex = {k:v for k,v in stock_lex.items() if len(k.split(' '))==1}

stock_lex_scaled = {}
for k, v in stock_lex.items():
    if v > 0:
        stock_lex_scaled[k] = v / max(stock_lex.values()) * 4
    else:
        stock_lex_scaled[k] = v / min(stock_lex.values()) * -4

# Loughran and McDonald
positive = []
with open('lexicon_data/lm_positive.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        positive.append(row[0].strip())
    
negative = []
with open('lexicon_data/lm_negative.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        entry = row[0].strip().split(" ")
        if len(entry) > 1:
            negative.extend(entry)
        else:
            negative.append(entry[0])
            
final_lex = {}
final_lex.update({word:2.0 for word in positive})
final_lex.update({word:-2.0 for word in negative})
final_lex.update(stock_lex_scaled)
final_lex.update(sia.lexicon)
sia.lexicon = final_lex

In [26]:
df = pd.read_csv('export_dataframe.csv')

In [27]:
df.head()

,Company,Article,Translation,Date,Negative,Neutral,Positive,Compound
0,Shell,Deze week begon er de aanleg van het zonnepark...,This week began the construction of the solar ...,27 May 2020 21:18:25 GMT,0.462,0.217,0.321,0.4804
1,Shell,Royal Dutch Shell (RDS) deint niet mee op de h...,Royal Dutch Shell (RDS) sways not count on rec...,27 May 2020 21:18:25 GMT,0.407,0.304,0.289,-0.0296
2,Shell,Shell wil meer geld voor stopzetten Groningen....,Shell wants more money to stop Groningen. The ...,27 May 2020 21:18:25 GMT,0.491,0.225,0.284,-0.5449
3,Shell,"Het gaat om Royal Dutch Shell (-0,6%), ASML (-...","It is Royal Dutch Shell (-0.6%), ASML (-0.6%),...",27 May 2020 21:18:25 GMT,0.194,0.479,0.328,0.4459
4,Shell,Daarin is afgesproken dat Shell en ExxonMobil ...,It was agreed that Shell and ExxonMobil would ...,27 May 2020 21:18:25 GMT,0.224,0.283,0.493,0.5648


In [5]:
from bs4 import BeautifulSoup
shell = feedparser.parse("https://www.google.com/alerts/feeds/11557311996690746829/885257184118668742")

In [19]:
a = []
b = []
c = []
d = []
e = []
f = []
g = []
for item in shell['items']:
    html = item['summary']
    soup = BeautifulSoup(html)
    text_parts = soup.findAll(text=True)
    text = ''.join(text_parts)
    translation = translator.translate(text)
    sentiment = sia.polarity_scores(translation.text)
    a.append(text)
    b.append(translation.text)
    c.append(shell['headers']['Date'][5:])
    d.append(sentiment['neg'])
    e.append(sentiment['neu'])
    f.append(sentiment['pos'])
    g.append(sentiment['compound'])

In [20]:
sia.polarity_scores(translation.text)['neg']

0.224

In [21]:
z = {'Article':a,'Translation':b,'Date':c,'Negative':d,'Neutral':e,'Positive':f,'Compound':d}
df= pd.DataFrame(z)

In [22]:
df.head()

,Article,Translation,Date,Negative,Neutral,Positive,Compound
0,Deze week begon er de aanleg van het zonnepark...,This week began the construction of the solar ...,27 May 2020 20:43:49 GMT,0.462,0.217,0.321,0.462
1,Royal Dutch Shell (RDS) deint niet mee op de h...,Royal Dutch Shell (RDS) sways not count on rec...,27 May 2020 20:43:49 GMT,0.407,0.304,0.289,0.407
2,Shell wil meer geld voor stopzetten Groningen....,Shell wants more money to stop Groningen. The ...,27 May 2020 20:43:49 GMT,0.491,0.225,0.284,0.491
3,"Het gaat om Royal Dutch Shell (-0,6%), ASML (-...","It is Royal Dutch Shell (-0.6%), ASML (-0.6%),...",27 May 2020 20:43:49 GMT,0.194,0.479,0.328,0.194
4,Daarin is afgesproken dat Shell en ExxonMobil ...,It was agreed that Shell and ExxonMobil would ...,27 May 2020 20:43:49 GMT,0.224,0.283,0.493,0.224


In [23]:
df.to_csv(r'export_dataframe.csv', index = False, header=True)


In [64]:
from googletrans import Translator
translator = Translator()
translation = translator.translate(text)
print(translation.text)

It was agreed that Shell and ExxonMobil would receive 90 million euros in compensation. Last week the Court ruled yet ...


In [61]:
text

'Daarin is afgesproken dat Shell en ExxonMobil 90 miljoen euro zouden krijgen ter compensatie. Vorige week oordeelde de Algemene Rekenkamer nog\xa0...'

In [7]:
print(shell['headers']['Date'][5:].replace(' GMT',''))

27 May 2020 18:31:08


In [17]:
b

['This week began the construction of the solar park Shell. The reconstruction of the site of the former gas purification (GZI) in Emmen ...',
 'Royal Dutch Shell (RDS) sways not count on recovering oil. Flag formation suggests verkoopruk at the Blue Chip. The price of a barrel ...',
 'Shell wants more money to stop Groningen. The compensation for the discontinuation of the gas creates dissatisfaction with Shell and ...',
 'It is Royal Dutch Shell (-0.6%), ASML (-0.6%), Unilever (-1.1%), relx (-1.0%) and DSM (-1.9%). The struggle for a working coronavirus vaccine. That we ...',
 'It was agreed that Shell and ExxonMobil would receive 90 million euros in compensation. Last week the Court ruled yet ...']

In [169]:
sentiment = sia.polarity_scores(b)
print(sentiment)

{'neg': 0.28, 'neu': 0.132, 'pos': 0.588, 'compound': 0.8096}


In [171]:
print(sentiment['compound'])

0.588


In [174]:
sentiment = sia.polarity_scores('chinese')

In [176]:
print(sentiment['compound'])

-0.0128
